[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/openlayer-python/blob/main/examples/tracing/openai/openai_agents_tracing.ipynb)

# OpenAI Agents SDK with Openlayer Tracing

This notebook demonstrates how to integrate **OpenAI Agents SDK** with **Openlayer** for comprehensive tracing and monitoring of multi-agent conversations.

## What you'll learn:
- How to set up OpenAI Agents SDK with Openlayer tracing
- How to create multiple agents with different roles
- How to implement handoffs between agents
- How to use function tools in agents
- How to monitor the complete conversation flow in Openlayer

## Requirements:
- OpenAI API key
- Openlayer API key and Inference Pipeline ID
- Internet connection for installing dependencies


## 1. Installation

First, let's install the required dependencies:


In [ ]:
# Install OpenAI Agents SDK and Openlayer
!pip install openai-agents openlayer

## 2. Environment Variables Setup

**⚠️ IMPORTANT**: Replace the placeholder values with your actual API keys:

### Required Environment Variables:
- **`OPENAI_API_KEY`**: Your OpenAI API key (get it from https://platform.openai.com/api-keys)
- **`OPENLAYER_API_KEY`**: Your Openlayer API key (get it from your Openlayer dashboard)
- **`OPENLAYER_INFERENCE_PIPELINE_ID`**: Your Openlayer inference pipeline ID (create one in your Openlayer dashboard)

### How to get these:
1. **OpenAI API Key**: Go to https://platform.openai.com/api-keys and create a new API key
2. **Openlayer API Key**: Log into your Openlayer dashboard and go to Settings → API Keys
3. **Inference Pipeline ID**: Create a new inference pipeline in your Openlayer dashboard and copy the ID


In [ ]:
import os

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"

# Set up Openlayer environment variables
os.environ["OPENLAYER_INFERENCE_PIPELINE_ID"] = "YOUR_OPENLAYER_INFERENCE_PIPELINE_ID_HERE"
os.environ["OPENLAYER_API_KEY"] = "YOUR_OPENLAYER_API_KEY_HERE"


## 3. Imports and Setup

Let's import all the necessary modules and set up logging for better debugging:


In [ ]:
import uuid
import random

# OpenAI Agents SDK imports
from agents import (
    Agent,
    Runner,
    ItemHelpers,
    ToolCallItem,
    HandoffOutputItem,
    MessageOutputItem,
    RunContextWrapper,
    ToolCallOutputItem,
    trace as agent_trace,
    handoff,
    function_tool,
    set_trace_processors,
)
from pydantic import BaseModel
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

# Openlayer integration
from openlayer.lib.integrations.openai_agents import OpenlayerTracerProcessor


## 4. Configure Openlayer Tracing

Now let's set up the Openlayer tracing processor. This will automatically capture all agent interactions and send them to Openlayer for monitoring:


In [ ]:
# Set up Openlayer tracing processor
set_trace_processors([
    OpenlayerTracerProcessor(
        service_name="airline_customer_service",
        version="1.0.0",
        environment="development"
    )
])

## 5. Define Context and Data Models

Let's define the context model that will be shared across all agents. This helps maintain conversation state:


In [ ]:
class AirlineAgentContext(BaseModel):
    """Context model to maintain conversation state across agents."""
    passenger_name: str | None = None
    confirmation_number: str | None = None
    seat_number: str | None = None
    flight_number: str | None = None


## 6. Create Function Tools

Function tools are reusable functions that agents can call to perform specific tasks. Let's create tools for FAQ lookup and seat updates:


In [ ]:
@function_tool(
    name_override="faq_lookup_tool",
    description_override="Lookup frequently asked questions."
)
async def faq_lookup_tool(question: str) -> str:
    """Tool to answer frequently asked questions about the airline."""
    if "bag" in question or "baggage" in question:
        return (
            "You are allowed to bring one bag on the plane. "
            "It must be under 50 pounds and 22 inches x 14 inches x 9 inches."
        )
    elif "seats" in question or "plane" in question:
        return (
            "There are 120 seats on the plane. "
            "There are 22 business class seats and 98 economy seats. "
            "Exit rows are rows 4 and 16. "
            "Rows 5-8 are Economy Plus, with extra legroom. "
        )
    elif "wifi" in question:
        return "We have free wifi on the plane, join Airline-Wifi"
    return "I'm sorry, I don't know the answer to that question."


@function_tool
async def update_seat(
    context: RunContextWrapper[AirlineAgentContext],
    confirmation_number: str,
    new_seat: str
) -> str:
    """
    Update the seat for a given confirmation number.

    Args:
        confirmation_number: The confirmation number for the flight.
        new_seat: The new seat to update to.
    """
    # Update the context based on the customer's input
    context.context.confirmation_number = confirmation_number
    context.context.seat_number = new_seat
    # Ensure that the flight number has been set by the incoming handoff
    assert context.context.flight_number is not None, "Flight number is required"
    return f"Updated seat to {new_seat} for confirmation number {confirmation_number}"

## 7. Create Hook Functions

Hooks are functions that run when specific events occur, such as agent handoffs. Let's create a hook for seat booking:


In [ ]:
async def on_seat_booking_handoff(context: RunContextWrapper[AirlineAgentContext]) -> None:
    """Hook that runs when handing off to the seat booking agent."""
    # Generate a random flight number when booking seats
    flight_number = f"FLT-{random.randint(100, 999)}"
    context.context.flight_number = flight_number

## 8. Create Specialized Agents

Now let's create our specialized agents. Each agent has a specific role and set of tools:


In [ ]:
# FAQ Agent - Handles frequently asked questions
faq_agent = Agent[AirlineAgentContext](
    name="FAQ Agent",
    handoff_description="A helpful agent that can answer questions about the airline.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are an FAQ agent. If you are speaking to a customer, you probably were transferred to from the triage agent.
    Use the following routine to support the customer.
    # Routine
    1. Identify the last question asked by the customer.
    2. Use the faq lookup tool to answer the question. Do not rely on your own knowledge.
    3. If you cannot answer the question, transfer back to the triage agent.""",
    tools=[faq_lookup_tool],
)

# Seat Booking Agent - Handles seat changes and updates
seat_booking_agent = Agent[AirlineAgentContext](
    name="Seat Booking Agent",
    handoff_description="A helpful agent that can update a seat on a flight.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    You are a seat booking agent. If you are speaking to a customer, you probably were transferred to from the triage agent.
    Use the following routine to support the customer.
    # Routine
    1. Ask for their confirmation number.
    2. Ask the customer what their desired seat number is.
    3. Use the update seat tool to update the seat on the flight.
    If the customer asks a question that is not related to the routine, transfer back to the triage agent. """,
    tools=[update_seat],
)

# Triage Agent - Routes customers to the appropriate specialized agent
triage_agent = Agent[AirlineAgentContext](
    name="Triage Agent",
    handoff_description="A triage agent that can delegate a customer's request to the appropriate agent.",
    instructions=(
        f"{RECOMMENDED_PROMPT_PREFIX} "
        "You are a helpful triaging agent. You can use your tools to delegate questions to other appropriate agents."
    ),
    handoffs=[
        faq_agent,
        handoff(agent=seat_booking_agent, on_handoff=on_seat_booking_handoff),
    ],
)

# Set up bidirectional handoffs (agents can return to triage)
faq_agent.handoffs.append(triage_agent)
seat_booking_agent.handoffs.append(triage_agent)


## 9. Create a Conversation Runner

Now let's create a function to run a conversation with our agents:


In [ ]:
async def run_conversation(user_input: str, conversation_id: str = None) -> str:
    """Run a single conversation turn with the agent system."""
    if conversation_id is None:
        conversation_id = uuid.uuid4().hex[:16]

    current_agent = triage_agent
    context = AirlineAgentContext()

    responses = []

    # Wrap the agent execution in a trace for Openlayer monitoring
    with agent_trace("Customer service", group_id=conversation_id):
        result = await Runner.run(current_agent, user_input, context=context)

        # Process the results
        for new_item in result.new_items:
            agent_name = new_item.agent.name

            if isinstance(new_item, MessageOutputItem):
                message = ItemHelpers.text_message_output(new_item)
                responses.append(f"{agent_name}: {message}")
            elif isinstance(new_item, HandoffOutputItem):
                handoff_msg = f"Handed off from {new_item.source_agent.name} to {new_item.target_agent.name}"
                responses.append(handoff_msg)
            elif isinstance(new_item, ToolCallItem):
                responses.append(f"{agent_name}: Calling a tool")
            elif isinstance(new_item, ToolCallOutputItem):
                responses.append(f"{agent_name}: Tool call output: {new_item.output}")
            else:
                responses.append(f"{agent_name}: {new_item.__class__.__name__}")

    return "\n".join(responses)


## 10. Test the Integration

Let's test our multi-agent system with different types of queries. Each conversation will be automatically traced and sent to Openlayer:

### Test 1: FAQ Query about Baggage


In [ ]:
# Test FAQ functionality
response = await run_conversation("What are the baggage restrictions?")
response


### Test 2: Seat Booking Request


In [ ]:
# Test seat booking functionality
response = await run_conversation("I want to change my seat")
response


### Test 3: Complex Multi-turn Conversation


In [ ]:
# Test a more complex interaction
response = await run_conversation("I need help with my flight. Can you tell me about the seats and also help me change mine?")
response
